In [ ]:
https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/788838/Homelessness_Models.pdf
    
Standard Score. The standard score (more commonly referred to as a z-score) 
is a very useful statistic because it (a) allows us to calculate the probability
of a score occurring within our normal distribution and (b) enables us to compare 
two scores that are from different normal distributions.


Comparing Z scores, who did relatively better!
The t test (also called Student’s T Test) compares two averages (means) and 
tells you if they are different from each other. 
The t test also tells you how significant the differences are; 
In other words it lets you know if those differences could have happened by chance.

https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3154788

In [67]:
To-do:
- examples of "apparently objective output" use SAT or LSAT score example and some others.
- Remove "ZFYA" from the alalyse feature dropdown,
- Move the description of " " to 'watch out for'
- Change the format of the "In your opinion should this data be used if the objective is to train an ML model which will reflect the selected worldview for 'race'?"
- Add another free text box to the very end of report?
- add info to the "watch out for" button in the intentional proxy box. 
- Add the data frame html to the file



1


In [215]:
%run ./FIR_Helper_Methods.ipynb
%run ./FIR_pre-process_data.ipynb
# import plotly.io as pio
# pio.renderers.default = 'iframe'
# http://homework.uoregon.edu/pub/class/es202/ztest.html#:~:text=Ultimately%2C%20the%20purpose%20of%20this,significantly%20different%20from%20one%20another.&text=Remember%2C%20statistically%20significant%20differences%20mean,that%20produce%20the%20other%20distribution.
# https://towardsdatascience.com/hypothesis-testing-in-machine-learning-using-python-a0dc89e169ce

<IPython.core.display.Javascript object>

Autosaving every 1000 seconds


# Audit and Transform the data for training

In [216]:
#For any rendering issue with plotly, eg browser freeze.. see https://github.com/jupyterlab/jupyter-renderers/issues/165
dpUI = data_pre_process_UI()
dpUI.render()



Box(children=(HTML(value='\n            <font style="font-family:sans-serif; font-size:15px;color:white;">\n  …

Box(children=(HTML(value='\n            <font style="font-family:sans-serif; font-size:15px;color:black;">\n  …

GridBox(children=(HTML(value='<h3><left>Upload the data file to be used to train/test/validate your ML model :…

GridBox(children=(FileUpload(value={}, accept='.csv', button_style='success', description='Upload', layout=Lay…

Label(value='  ', layout=Layout(width='100%'))

...

Label(value='  ', layout=Layout(width='100%'))

HTML(value='<h3><left>Scale/Normalise numeric input features: </left></h3>')

GridBox(children=(HTML(value='\n            <font style="font-family:sans-serif; font-size:15px;color:grey;">\…

HTML(value='<h3><left>Save transformed data to file.</left></h3>')

Output()

In [212]:
display (dpUI.non_protected_feature_set_df.head())

,LSAT,UGPA,region_first,sander_index
0,39.0,3.1,GL,0.782738
1,36.0,3.0,GL,0.735714
2,30.0,3.1,MS,0.670238
3,39.0,2.2,NE,0.697024
4,37.0,3.4,GL,0.786310


# Apply Any further transformations not supported by the GUI
- Do not delete any of the columns created by the GUI pricess

# Review the transformed dataset (input features and label data)

once the data has been transformed and the transformed data is saved we can now train a model

We will read the transformed data frame and the data_summary which contains details of the transform and information on which features to use to train the model.


In [34]:
helper_method = helper_methods()
pickle_path = "/Users/aideenf/Documents/GitHub/Thesis_Aideen/Main/Data/transformed_data_2.csv_summary.pickle"
data_frame_path = "/Users/aideenf/Documents/GitHub/Thesis_Aideen/Main/Data/transformed_data_2.csv"

# Reload the file
data_frame, data_summary = helper_method.reload_data (pickle_path, 
                                                      data_frame_path, 
                                                      print_report = True)
# Reload the file

Number of samples in dataset:  21791
Output to predict:  ZFYA
Output is continuous:  True
Output is binary:  False
Num of unique outputs:  568
Min output:  -3.35
Max output:  3.48
High output range is positive?  True

Protected input features ['gender', 'race_Amerindian', 'race_Asian', 'race_Black', 'race_Hispanic', 'race_Mexican', 'race_Other', 'race_Puertorican', 'race_White']
Other input features ['LSAT', 'UGPA', 'sander_index', 'first_pf', 'region_first_FW', 'region_first_GL', 'region_first_MS', 'region_first_MW', 'region_first_Mt', 'region_first_NE', 'region_first_NG', 'region_first_NW', 'region_first_PO', 'region_first_SC', 'region_first_SE']

Summary of Data transformation per input feature:
************
race


************
gender


************
region_first


************
first_pf


AttributeError: 'file_data' object has no attribute 'proxyAttributes'

# Review the training/test data to set positive and negative(Binary) output ranges for Logistic Regression.

Select the decision boundary between a positive and negative outcome of a continuous target variable, for the purpose of logistic regression.

In [ ]:
helper_method.view_protected(data_summary.protected_x, 
                             data_summary.y_value, 
                             data_frame, _w=600, _h=600, 
                             y_high_positive = data_summary.HIGH_RANGE_POSITIVE,
                             persist_impact_col = True,
                             output_type_binary = data_summary.Y_BINARY)


In [ ]:
display (data_frame.head())

# Prepare transformed data for model training
- using the data summary as input the y value and appropriate X values will be returned
- Specify here if wany to imclude the protected data in the training or not. include_protected = True or False
- Right now we can only specify to include all protected, or none(True/False). If you only want to include some protected input features use include_protected = True, then drop the columns from the returned 'X' AND also drop the column from array data_summary.protected_column_list

In [ ]:
use_transformed = True
if use_transformed == True:
    data_summary.y_value = "Transformed_" + data_summary.y_value

'''library Helper method
input the data_frame, the data_summary generated from reviewing the data or set independently.
'''
X, y, feature_columns = helper_method.get_train_test_split_data(data_frame, 
                                                                data_summary, 
                                                                include_protected = True
                                                               )
'''************'''


#  Create train/test split + SkLearn Logistic Regression+Return probability score. 
once we have created the model we have the logistic regression prediction probability, so that each individual I has a probability pᵢ to fall into category A, and q = 1-p to fall into category B

 Less formally, it can be thought of as a model for the set of possible outcomes of any single data row that asks a yes–no question. Such questions lead to outcomes that are boolean-valued: a single bit whose value is success/yes/true/one with probability p and failure/no/false/zero with probability q = 1 -p. All successful outcomes may then be ranked by probability score inorder to obtain an ordered list.
 
 Logistic regression is a machine learning model that uses a logistic function to model a binary dependent variable. Although there are better models in terms of performance (a.o. boosting methods, random forest, deep neural networks), logistic regression provides coefficients which are very easy to interpret. As this is an important feature for demonstration purposes, logistic regression is used here.



In [ ]:
#from sklearn.linear_model import LogisticRegressionCV
# clf = LogisticRegressionCV(cv=5, solver = "liblinear", random_state=0, max_iter = 5000).fit(X_train, y_train)
# """
# Set up the logistic regression model
# """
from sklearn.metrics import classification_report
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=0, 
                                                    test_size = 0.20)

logistic_reg_model = LogisticRegression(solver = "liblinear", verbose = 1, random_state=0, max_iter = 1000).fit(X_train[feature_columns], y_train)
print ("Train Accuracy score: ", logistic_reg_model.score(X_train[feature_columns], y_train))
logistic_reg_y_train_pred = logistic_reg_model.predict(X_train[feature_columns])
logistic_reg_y_train_pred_prob = logistic_reg_model.predict_proba(X_train[feature_columns])
print(classification_report(y_train, logistic_reg_y_train_pred))
print ("")
print ("Test Accuracy score: ", logistic_reg_model.score(X_test[feature_columns], y_test))
logistic_reg_y_test_pred = logistic_reg_model.predict(X_test[feature_columns])
logistic_reg_y_test_pred_prob = logistic_reg_model.predict_proba(X_test[feature_columns])
print(classification_report(y_test, logistic_reg_y_test_pred))


def get_probability_y_actual(y_actual, y_prob):
    y_actual_prob = []
    y_prob_1 = []
    for y_a, y_p, i in zip(y_actual, y_prob, range(len(y_actual))):
        if y_a == 0:
            y_actual_prob.append(y_prob[i][0])
            y_prob_1.append(y_prob[i][1])
        else:
            y_actual_prob.append(y_prob[i][1])
            y_prob_1.append(y_prob[i][1])
    return y_actual_prob, y_prob_1


y_train_pred_prob, y_train_pred_prob_1 = get_probability_y_actual(y_train, logistic_reg_y_train_pred_prob)
y_test_pred_prob, y_test_pred_prob_1 = get_probability_y_actual(y_test, logistic_reg_y_test_pred_prob)


helper_method.visualise_RMSE_model_eval(y_train, y_test, y_train_pred_prob, y_test_pred_prob)

# Analise with Aequitas

https://towardsdatascience.com/a-tutorial-on-fairness-in-machine-learning-3ff8ba1040cb
https://arxiv.org/pdf/1908.00176.pdf
https://5harad.com/papers/fair-ml.pdf


In [ ]:
# print (data_summary.renamed)
# print (data_summary.referenceNames)
# print(data_summary.encoding_dict)

#Retrieve only the protected attributes
focus_on_cols = helper_method.get_protected(data_summary)

'''For additional granularity retrieve all versions of the protected attributes
(before encoding and before values merged)'''
# focus_on_cols_benc = helper_method.get_protected_before_transform(data_summary)


# data_summary.non_protected_x
# data_summary.y_value

helper_method.view_aequitas_fairness_metrics(data_summary.y_value ,
                                       focus_on_cols,
                                       X_train, 
                                       y_train, 
                                       logistic_reg_y_train_pred,
                                       _w=600, _h=600, 
                                       y_high_positive = True,
                                        )

# Shap interpretability
https://shap.readthedocs.io/en/latest/#
https://towardsdatascience.com/explain-any-models-with-the-shap-values-use-the-kernelexplainer-79de9464897a
https://github.com/Rakeshsuku/Medium-Blog/blob/master/Kernel_SHAP.ipynb
https://github.com/slundberg/shap/blob/master/README.md


In [ ]:
explainer, shap_values, x = helper_method.prepare_to_shap(X_train[feature_columns], 500, save_to_path = './')

In [ ]:
shap_path = "/Users/aideenf/Documents/GitHub/Thesis_Aideen/Main"
# Reload the file
shap_values, explainer, x =  helper_method.reload_shap_data (shap_path)
# Reload the file

display (x.head())

In [ ]:
helper_method.shap_analysis(shap_values, explainer, x, data_summary)

# data_summary.protected_x 
# data_summary.non_protected_x
# data_summary.y_value
# y_high_positive = data_summary.HIGH_RANGE_POSITIVE
# persist_impact_col = True
# output_type_binary = data_summary.Y_BINARY



# Ranking Fairness
We will rank the results based on the probability of a positive outcome.

Notes "Fairness is a complex concept, with a number of different possible definitions. 
We consider group fairness with respect to membership in a protected group, based, 
for example, on minority race or underrepresented gender, where group membership is 
readily ascertained by looking at an attribute value. For a given rank cut-off point k, 
we wish to ensure that the number of protected group members ranked among the top-$k$ is 
proportional to their representation in the entire population, or to their desired proportion
in the output (as is the case in affirmative action interventions)."


To-Do
#Create new protected attributes based on a combination of the other protected 
            #attributes so for example if you belong to the protected Group A of Attribute A
            #and protected grouo B of attribute B then you are in a double protected group
            #In order to achieve this we must provide the possibility of identifying 
            #the protected group for each protected attribute.
            
            #df['multi_protected'] = df.apply(lambda row: row.a and row.b, axis=1) 

In [ ]:
y_pred_prob = y_train_pred_prob
y_pred_prob_1 =  y_train_pred_prob_1
_y = y_train
_X = X_train

pd.set_option("display.precision", 10)
y_predicted_df = pd.DataFrame(y_pred_prob_1).rename(columns={0: "Predicted"})


df_for_ranking = _X.reset_index(drop=True)
df_for_ranking = df_for_ranking.merge(y_predicted_df, left_index=True, right_index=True)
ranked_df = df_for_ranking.sort_values(by ='Predicted',  ascending=False )
del df_for_ranking
display(ranked_df.head(5))
print (ranked_df.shape)
print ("High ranking has positive outcome: ", data_summary.HIGH_RANGE_POSITIVE)


In [ ]:
display (HTML('''<b>Values saved in data_summary</b> <br>  
        data_summary.renamed <br>
        data_summary.referenceNames<br>
        data_summary.encoding_dict<br>
        data_summary.df_url<br>
        data_summary.y_value<br>
        data_summary.Y_BINARY<br>
        data_summary.Y_CONTINUOUS<br>
        data_summary.HIGH_RANGE_POSITIVE<br>
        data_summary.protected_x<br>
        data_summary.non_protected_x<br>
        data_summary.all_columns_in_x<br>
        data_summary.proxyAttributes<br>
        data_summary.MLAttributes<br>
        data_summary.html'''))

In [ ]:
gc.collect()

In [ ]:
%run ./FIR_Review_Fairness_After_Ranking.ipynb

ranked_fairness = review_ranked_fairness_UI(ranked_df=ranked_df, 
                              protected_attributes=data_summary.protected_x, 
                              renamed_groups=data_summary.renamed,
                              encoded_groups = data_summary.encoding_dict,
                              group_names = data_summary.referenceNames,
                              high_ranking_positive=data_summary.HIGH_RANGE_POSITIVE, 
                              )

ranked_fairness.render()

In [ ]:
%reset out

In [ ]:
get_ipython().events.callbacks

# SkLearn Linear regression model for Ranking

Remember, a linear regression model in two dimensions is a straight line; in three dimensions it is a plane, and in more than three dimensions, a hyperplane. Here we have multiple dimensions or input variables so our model will be a “multiple linear regression” or "multivariate linear regression". It would be possible to predict the exact FYA by making this a regression task.

We should convert a pandas DataFrame into a NumPy array in order to train a machine learning model on it. 
Being careful to remove the target variable from the data before training a model.

Note:Including the output variable, or some derived property of the output variable, into the feature representation is a very common mistake in building supervised machine learning models.

NOTE: To train with both protected and non protected use data_summary.protected_x + data_summary.non_protected_x columns of the transformed data frame.

In [ ]:
from sklearn.metrics import classification_report

linreg = LinearRegression(fit_intercept=True)
linreg.fit(X_train, y_train)
print (linreg.score(X_train, y_train))
y_pred = linreg.predict(X_train)
print (y_train[:20])
print (" ")
print (y_pred[:20])
classification_report(y_train, y_pred)

# Keras Neural Network 

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense



counts = y_train.value_counts()
print (counts)
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(y_train)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

model = keras.Sequential(
    [
        keras.layers.Dense(
            256, activation="relu", input_shape=(X_train[feature_columns].shape[-1],)
        ),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)

callbacks = [keras.callbacks.ModelCheckpoint("model_at_epoch_{epoch}.h5")]
class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(
    X_train[feature_columns],
    y_train,
    batch_size=2048,
    epochs=50,
    verbose=2,
    callbacks=callbacks,
    validation_data=(X_test[feature_columns], y_test),
    class_weight=class_weight,
);
y_pred = model.predict(X_train[feature_columns])
classification_report(y_train, y_pred)


# ######################ALT NEURAL NW

# print (X_train.columns)
# print (y_train)
# # Neural network with keras tutorial
# neural_model = Sequential()
# neural_model.add(Dense(2, input_dim=len(X_train.columns), activation='tanh'))
# neural_model.add(Dense(2, activation='tanh'))
# neural_model.add(Dense(1, activation='sigmoid'))
# neural_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# # fit the keras model on the dataset
# neural_model.fit(X_train, y_train, epochs=150, batch_size=10)
# # evaluate the keras model
# _, accuracy = neural_model.evaluate(X_train, y_train)
# print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
print (y_pred[:5])
print(len(np.unique(np.array(y_pred))))

In [ ]:
for i in range(len(y_pred)):
    if y_pred[i] >.5:
        y_pred[i] = 1
    else:
        y_pred[i] = 0 
print (classification_report(y_train, y_pred))

In [ ]:


X1 = stats.norm.rvs(loc=10, scale=2, size=30, random_state=0)
X2 = stats.norm.rvs(loc=10, scale=2, size=30, random_state=1)
X3 = stats.norm.rvs(loc=14, scale=2, size=30, random_state=2)

print(stats.ttest_ind(X1, X2))
print(stats.ttest_ind(X1, X3))